In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np

path = "/content/drive/MyDrive/colab/CORE_Proyecto_i/retail_sales_dataset.csv"
df = pd.read_csv(path)

Mounted at /content/drive


In [2]:
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


##Transformación de Datos
Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.

In [27]:
## Clasificaremos la edad de los clientes para tener una idea de que generacion hace mas ventas

def clasificar_edad(age):
    if 18 <= age <= 30:
        return "Young Adult"
    elif 31 <= age <= 45:
        return "Middle Adult"
    elif 46 <= age <= 59:
        return "Mature Adult"
    else:
        return "Elderly"

## Creamos la nueva columna
df["Age segment"] = df['Age'].apply(clasificar_edad)
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Segmento de edad,Clasificacion ventas,Age segment
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,Middle Adult,Baja,Middle Adult
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,Young Adult,Media,Young Adult
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,Mature Adult,Baja,Mature Adult
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,Middle Adult,Baja,Middle Adult
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,Young Adult,Baja,Young Adult


In [28]:
df.value_counts("Age segment")

## Gracias a la columna que creamos podemos ver que los "Mature Adult" hacen más transacciones

,count
Age segment,
Mature Adult,309
Middle Adult,303
Young Adult,273
Elderly,115


In [29]:
df.groupby("Age segment")["Total Amount"].mean().sort_values(ascending=False)

## Sin embargo, tambien podemos ver que a pesar del resultado anterior, quienes han comprado el mayor monto (en promedio) son los "Young Adult"

,Total Amount
Age segment,
Young Adult,486.978022
Middle Adult,468.498350
Mature Adult,441.051780
Elderly,389.695652


Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

teniendo en cuenta que la venta menor es de 25 y la mayor 2000 la clasificaremos:

25-700

701-1500

1501-2000

In [30]:
def clasificar_ventas(amount):
    if 25 <= amount <= 700:
        return "Baja"
    elif 701 <= amount <= 1500:
        return "Media"
    else:
        return "Alta"

df["Sales Level"] = df["Total Amount"].apply(clasificar_ventas)
df.head()

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount,Segmento de edad,Clasificacion ventas,Age segment,Sales Level
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,Middle Adult,Baja,Middle Adult,Baja
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,Young Adult,Media,Young Adult,Media
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,Mature Adult,Baja,Mature Adult,Baja
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,Middle Adult,Baja,Middle Adult,Baja
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,Young Adult,Baja,Young Adult,Baja


###Agrupación y Agregación
Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.

Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [44]:
df.groupby("Product Category")["Price per Unit"].mean()

,Price per Unit
Product Category,
Beauty,184.055375
Clothing,174.287749
Electronics,181.900585


In [36]:
df.groupby(["Gender", "Age segment"])["Total Amount"].sum()

### En esta tabla podemos ver cuanto compra cada genero y en que rango de edad

Gender  Age segment 
Female  Elderly         21090
        Mature Adult    70430
        Middle Adult    71510
        Young Adult     69810
Male    Elderly         23725
        Mature Adult    65855
        Middle Adult    70445
        Young Adult     63135
Name: Total Amount, dtype: int64

In [37]:
df.groupby(["Gender", "Product Category"])["Total Amount"].sum()

## En esta tabla podemos ver en que producto gasta cada genero

Gender  Product Category
Female  Beauty              74830
        Clothing            81275
        Electronics         76735
Male    Beauty              68685
        Clothing            74305
        Electronics         80170
Name: Total Amount, dtype: int64

In [41]:
df.groupby(["Product Category", "Age segment"])["Total Amount"].sum().unstack()

## En esta tabla podemos ver en que segmento de edad se vende mas un producto

Age segment,Elderly,Mature Adult,Middle Adult,Young Adult
Product Category,,,,
Beauty,5620,46970,45215,45710
Clothing,15130,44415,47365,48670
Electronics,24065,44900,49375,38565


### Análisis Personalizado con apply
Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.

Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [54]:
des_std = df.groupby(["Product Category"])["Total Amount"].std()
des_std

,Total Amount
Product Category,
Beauty,563.612788
Clothing,550.695917
Electronics,567.540150
